In [1]:
import os
os.chdir("../")
os.getcwd()

'/home/izam/coding/hackathon'

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_data_file: Path
    source_info_file: Path
    local_data_name: str

In [3]:
from gemInsights.constants import *
from gemInsights.utils.common import read_yaml, create_directories

In [4]:
#udpate configuration manager
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        credentials_file_path = CREDENTIALS_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.credentials = credentials_file_path
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_data_file=config.source_data_file,
            source_info_file=config.source_info_file,
            local_data_name=config.local_data_name,
        )

        return data_ingestion_config

In [5]:
import os
from gemInsights import logger
import pandas as pd
from gemInsights.utils.common import load_json, save_json

In [6]:
# components
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def load_data(self):
        data_path = Path(self.config.source_data_file)
        df = pd.read_csv(data_path)
        logger.info(f"loaded the dataframe with shape {df.shape}")

        df.to_csv(os.path.join(self.config.root_dir, self.config.local_data_name))
        logger.info(f"saved the data in {os.path.join(self.config.root_dir, self.config.local_data_name)}")

    def load_info(self):
        json_file = load_json(path=Path(self.config.source_info_file))
        data_name, _ = os.path.splitext(self.config.local_data_name)
        save_json(path=Path(os.path.join(self.config.root_dir, data_name+"_info.json")), data=json_file)

    def store_data(self):
        """
        code for store the data into the cloud
        """

In [7]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.load_data()
    data_ingestion.load_info()
    data_ingestion.store_data()
except Exception as e:
    raise e

[2023-12-21 13:24:17,901: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-12-21 13:24:17,903: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-21 13:24:17,903: INFO: common: created directory at: artifacts]
[2023-12-21 13:24:17,904: INFO: common: created directory at: artifacts/data_ingestion]
[2023-12-21 13:24:17,908: INFO: 1086895689: loaded the dataframe with shape (891, 12)]
[2023-12-21 13:24:17,914: INFO: 1086895689: saved the data in artifacts/data_ingestion/data.csv]
[2023-12-21 13:24:17,915: INFO: common: json file loaded succesfully from: streamlit_files/additional_data.json]
[2023-12-21 13:24:17,915: INFO: common: json file saved at: artifacts/data_ingestion/data_info.json]
